

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/CLASSIFICATION_RU_TOXIC.ipynb)






## **Toxic content classifier for Russian**


## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

## 2. Start Spark Session

In [3]:
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 4.2.8


## 3. Sample Examples texts.

In [4]:
TEXT_LIST = ["""но ведь многие бросают свою половину из-за бездетности.А он нет-значит любил.Конечно-он при жизни мог завещание сделать.""","""Ненавижу тебя, идиот.""","""Согласна. Сама думаю, какую площадь приспособить и гараж ну никак не вписывается.""","""Вот ты хуйню какую-то на пикабу постишь, а в это время враг боевые навыки совершенствует""","""У нас почему-то только по холодам такие мероприятия. Так что к осени""","""Никчёмный народец получивший славу только потому что Батый не вырезал их сразу а ставил первыми шеренгами или живым щитом перед своими же войсками. Выживаемость ноль""","""Не думаю что его выгодно делать ,так как я видел эти телефоны новые с рук по 5к""","""И хитлара, и сралина считаются одинаково усатыми идиотами."""]

## 4. Define Spark NLP pipeline


In [5]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

sequenceClassifier = BertForSequenceClassification \
      .pretrained('bert_sequence_classifier_toxicity', 'ru') \
      .setInputCols(['token', 'document']) \
      .setOutputCol('class_')

russian_pipeline = Pipeline(
    stages=[
        document_assembler, 
        tokenizer, 
        sequenceClassifier])

bert_sequence_classifier_toxicity download started this may take some time.
Approximate size to download 634.3 MB
[OK!]


## 5. Run the pipeline

In [6]:
df = spark.createDataFrame(TEXT_LIST, StringType()).toDF("text")
result = russian_pipeline.fit(df).transform(df)

## 6. Visualize results


In [7]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("document"),
              F.expr("cols['1']").alias("class")).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|document                                                                                                                                                             |class  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|но ведь многие бросают свою половину из-за бездетности.А он нет-значит любил.Конечно-он при жизни мог завещание сделать.                                             |neutral|
|Ненавижу тебя, идиот.                                                                                                                                                |toxic  |
|Согласна. Сама думаю, какую площадь приспособить и гараж ну никак не вписывается.                                      